In [1]:
from fmm import Network,NetworkGraph,FastMapMatch,FastMapMatchConfig,UBODT,GPSConfig,ResultConfig
import pandas as pd

In [2]:
#load network data and graph
network = Network("../osmnx_example/stockholm/edges.shp","fid", "u", "v")
print("Nodes {} edges {}".format(network.get_node_count(),network.get_edge_count()))
graph = NetworkGraph(network)

[2025-02-19 15:18:15.257] [info] [network.cpp:72] Read network from file ../osmnx_example/stockholm/edges.shp
Nodes 649 edges 1308
[2025-02-19 15:18:15.303] [info] [network.cpp:172] Number of edges 1308 nodes 649
[2025-02-19 15:18:15.303] [info] [network.cpp:173] Field index: id 15 source 12 target 13
[2025-02-19 15:18:15.304] [info] [network.cpp:176] Read network done.
[2025-02-19 15:18:15.306] [info] [network_graph.cpp:17] Construct graph from network edges start
[2025-02-19 15:18:15.306] [info] [network_graph.cpp:30] Graph nodes 649 edges 1308
[2025-02-19 15:18:15.306] [info] [network_graph.cpp:31] Construct graph from network edges end


### Precompute an UBODT file

**This step can be skipped if you already created one UBODT file.**

In [3]:
from fmm import UBODTGenAlgorithm
ubodt_gen = UBODTGenAlgorithm(network,graph)
status = ubodt_gen.generate_ubodt("../osmnx_example/stockholm//ubodt.txt", 4, binary=False, use_omp=True)
print(status)

Status: success
Time takes 0.671 seconds

[2025-02-19 15:18:15.322] [info] [ubodt_gen_algorithm.cpp:76] Start to generate UBODT with delta 4
[2025-02-19 15:18:15.323] [info] [ubodt_gen_algorithm.cpp:77] Output format csv
[2025-02-19 15:18:15.393] [info] [ubodt_gen_algorithm.cpp:105] Progress 64 / 649
[2025-02-19 15:18:15.461] [info] [ubodt_gen_algorithm.cpp:105] Progress 128 / 649
[2025-02-19 15:18:15.527] [info] [ubodt_gen_algorithm.cpp:105] Progress 192 / 649
[2025-02-19 15:18:15.590] [info] [ubodt_gen_algorithm.cpp:105] Progress 256 / 649
[2025-02-19 15:18:15.659] [info] [ubodt_gen_algorithm.cpp:105] Progress 320 / 649
[2025-02-19 15:18:15.724] [info] [ubodt_gen_algorithm.cpp:105] Progress 384 / 649
[2025-02-19 15:18:15.788] [info] [ubodt_gen_algorithm.cpp:105] Progress 448 / 649
[2025-02-19 15:18:15.855] [info] [ubodt_gen_algorithm.cpp:105] Progress 512 / 649
[2025-02-19 15:18:15.927] [info] [ubodt_gen_algorithm.cpp:105] Progress 576 / 649
[2025-02-19 15:18:15.982] [info] [ubodt_ge

### FMM model creation and map matching

In [ ]:
ubodt = UBODT.read_ubodt_csv("../osmnx_example/stockholm/ubodt.txt")
model = FastMapMatch(network,graph,ubodt)

# configuration parameters
k = 4
radius = 0.4
gps_error = 0.5
fmm_config = FastMapMatchConfig(k,radius,gps_error)

input_config = GPSConfig()
input_config.file = "../osmnx_example/stockholm/trips.csv"
input_config.id = "id"

result_config = ResultConfig()
result_config.file = "../osmnx_example/stockholm/mr.txt"
result_config.output_config.write_opath = True
print(result_config.to_string())

matched_df = pd.read_csv("../osmnx_example/stockholm/mr.txt", delimiter=";")
print(matched_df.head())


[2025-02-19 15:25:52.048] [info] [ubodt.cpp:208] Reading UBODT file (CSV format) from ../osmnx_example/stockholm/ubodt.txt
[2025-02-19 15:25:52.298] [info] [ubodt.cpp:243] Finish reading UBODT with rows 378341
Result file : ../osmnx_example/stockholm/mr.txt
Output fields: opath cpath mgeom 
   id                              opath  \
0   2          1226,817,1278,998,582,297   
1   1  37,957,984,203,1263,545,1059,1188   

                                               cpath  \
0  1226,1266,1275,817,517,1278,340,998,493,582,65...   
1  37,289,477,254,957,558,407,395,185,134,988,984...   

                                               mgeom  
0  LINESTRING(18.0619967958 59.3372556566,18.0617...  
1  LINESTRING(18.0551131103 59.3355378738,18.0550...  


In [19]:
from shapely.wkt import loads
from shapely.geometry import mapping
import pandas as pd
import json
from ipyleaflet import Map, GeoJSON, WidgetControl
import ipywidgets as widgets

# Load matched and original data
matched_df = pd.read_csv("../osmnx_example/stockholm/mr.txt", delimiter=";")
original_df = pd.read_csv("../osmnx_example/stockholm/trips.csv", delimiter=";")

# Merge original and matched trajectories by 'id'
merged_df = pd.merge(matched_df[['id', 'mgeom']], original_df[['id', 'geom']], on='id')

# Initialize lists for GeoJSON
original_geojson_list = []
matched_geojson_list = []

for _, row in merged_df.iterrows():
    # Convert original and matched geometry to GeoJSON
    original_geom = loads(row["geom"])  # 'geom' is the column for original trajectories
    matched_geom = loads(row["mgeom"])  # 'mgeom' is the column for matched trajectories
    
    original_geojson_list.append({
        "type": "Feature",
        "geometry": mapping(original_geom),
        "properties": {"id": row["id"], "type": "Original"}
    })
    
    matched_geojson_list.append({
        "type": "Feature",
        "geometry": mapping(matched_geom),
        "properties": {"id": row["id"], "type": "Matched"}
    })

# Create GeoJSON FeatureCollection for both original and matched
geojson_data = {"type": "FeatureCollection", "features": original_geojson_list + matched_geojson_list}

# Save to file (optional)
with open("merged_trajectories.geojson", "w") as f:
    json.dump(geojson_data, f)

# Initialize the map
m = Map(center=(59.34, 18.06), zoom=13)

# Add both original and matched layers with different colors
original_layer = GeoJSON(name="Original Trajectories", data={"type": "FeatureCollection", "features": original_geojson_list},
                         style={"color": "blue", "weight": 3})
matched_layer = GeoJSON(name="Matched Trajectories", data={"type": "FeatureCollection", "features": matched_geojson_list},
                        style={"color": "orange", "weight": 3})

m.add_layer(original_layer)
m.add_layer(matched_layer)

# Display the map
m


Map(center=[59.34, 18.06], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_o…